# **Import Required Libraries**

In [0]:
import pandas as pd
import numpy as np
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import tensorflow as tf
from tensorflow import keras

# **Load Data**

In [0]:
drive.mount('/content/drive') #connect to your gdrive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Big Data Project Files/appDF_final.csv") #load data

**The dataframe df: all tables one hot encoded, aggregated, joined, manual features included (may contain nulls)**

In [0]:
df.head() 

,Unnamed: 0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,REGION_RATING_CLIENT,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,EXT_SOURCE_2,EXT_SOURCE_3,YEARS_BEGINEXPLUATATION_AVG,FLOORSMAX_AVG,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,...,IN_CLIENT_IN_LOAN_NUM_INSTALMENT_NUMBER_mean_min,IN_CLIENT_IN_LOAN_NUM_INSTALMENT_VERSION_sum_min,IN_CLIENT_IN_LOAN_NUM_INSTALMENT_VERSION_sum_mean,IN_CLIENT_IN_LOAN_AMT_PAYMENT_min_min,IN_CLIENT_IN_LOAN_AMT_PAYMENT_min_mean,IN_CLIENT_IN_LOAN_AMT_PAYMENT_mean_min,IN_CLIENT_IN_LOAN_AMT_INSTALMENT_max_min,IN_CLIENT_IN_LOAN_AMT_INSTALMENT_max_mean,IN_CLIENT_IN_LOAN_AMT_PAYMENT_sum_min,IN_CLIENT_IN_LOAN_AMT_PAYMENT_sum_mean,CASH_CLIENT_CASH_LOAN_MONTHS_BALANCE_sum_min,CASH_CLIENT_CASH_LOAN_INSTALLMENTS_PAID_min_min,CASH_CLIENT_CASH_LOAN_SK_DPD_DEF_min_mean,CASH_CLIENT_CASH_LOAN_NAME_CONTRACT_STATUS_Canceled_mean_min,CASH_CLIENT_CASH_LOAN_NAME_CONTRACT_STATUS_Amortized debt_mean_min,CASH_CLIENT_CASH_LOAN_NAME_CONTRACT_STATUS_XNA_mean_mean,CASH_CLIENT_CASH_LOAN_NAME_CONTRACT_STATUS_Canceled_mean_mean,CASH_CLIENT_CASH_LOAN_NAME_CONTRACT_STATUS_Demand_mean_min,CASH_CLIENT_CASH_LOAN_NAME_CONTRACT_STATUS_Approved_mean_min,CASH_CLIENT_CASH_LOAN_NAME_CONTRACT_STATUS_Approved_mean_mean,CASH_CLIENT_CASH_LOAN_NAME_CONTRACT_STATUS_Returned to the store_mean_min,CASH_CLIENT_CASH_LOAN_SK_DPD_DEF_mean_min,CASH_CLIENT_CASH_LOAN_SK_DPD_mean_min,CASH_CLIENT_CASH_LOAN_INSTALLMENTS_PAID_min_mean,CASH_CLIENT_CASH_LOAN_INSTALLMENTS_PAID_min_max,CASH_CLIENT_CASH_LOAN_SK_DPD_DEF_mean_mean,CASH_CLIENT_CASH_LOAN_SK_DPD_DEF_sum_mean,CASH_CLIENT_CASH_LOAN_NAME_CONTRACT_STATUS_Returned to the store_mean_mean,CASH_CLIENT_CASH_LOAN_NAME_CONTRACT_STATUS_Signed_mean_min,CASH_CLIENT_CASH_LOAN_NAME_CONTRACT_STATUS_Signed_mean_mean,CASH_CLIENT_CASH_LOAN_CNT_INSTALMENT_FUTURE_min_min,CASH_CLIENT_CASH_LOAN_CNT_INSTALMENT_FUTURE_min_mean,CASH_CLIENT_CASH_LOAN_NAME_CONTRACT_STATUS_Completed_mean_min,CASH_CLIENT_CASH_LOAN_NAME_CONTRACT_STATUS_Completed_mean_mean,CASH_CLIENT_CASH_LOAN_NAME_CONTRACT_STATUS_Active_mean_min,CASH_CLIENT_CASH_LOAN_INSTALLMENTS_PAID_mean_mean,CASH_CLIENT_CASH_LOAN_CNT_INSTALMENT_FUTURE_mean_min,CASH_CLIENT_CASH_LOAN_CNT_INSTALMENT_FUTURE_mean_mean,CASH_CLIENT_CASH_LOAN_CNT_INSTALMENT_min_min,CASH_CLIENT_CASH_LOAN_CNT_INSTALMENT_min_mean
0,0,100002,1.0,0.0,202500.0,406597.5,24700.5,0.018801,-9461.0,-637.0,-3648.0,-2120.0,1.0,1.0,0.0,1.0,1.0,0.0,2.0,10.0,0.0,0.0,0.0,0.0,0.262949,0.139376,0.9722,0.0833,0.0149,2.0,2.0,-1134.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,20.0,20.000000,9251.775,9251.775,11559.247105,53093.745,53093.745,219625.695,219625.695,-190.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,6.0,6.000000,0.00,0.000000,1.000000,9.000000,15.000000,15.000000,24.0,24.000000
1,1,100003,0.0,0.0,270000.0,1293502.5,35698.5,0.003541,-16765.0,-1188.0,-1186.0,-291.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.622246,NaN,0.9851,0.2917,0.0714,1.0,0.0,-828.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.5,6.0,8.666667,6662.970,56431.860,6731.115000,6737.310,210713.445,80773.380,539621.550,-858.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.333333,0.00,0.083333,0.875000,4.125000,2.625000,5.666667,6.0,8.333333
2,2,100004,0.0,0.0,67500.0,135000.0,6750.0,0.010032,-19046.0,-225.0,-4260.0,-2531.0,1.0,1.0,1.0,1.0,1.0,0.0,2.0,9.0,0.0,0.0,0.0,0.0,0.555912,0.729567,NaN,NaN,NaN,0.0,0.0,-815.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,4.0,4.000000,5357.250,5357.250,7096.155000,10573.965,10573.965,21288.465,21

In [0]:
df.shape #check dim

(307511, 564)

# **Check proportion of labels**

In [0]:
ones = df[df['TARGET']==1]
ones.shape #24825 1's

(24825, 564)

In [0]:
zeros = df[df['TARGET']==0]
zeros.shape #282686 0's

(282686, 564)

In [0]:
frac=len(ones)/len(zeros) #only 8.7% are 1's - rest are 0's
frac

0.08781828601345662

In [0]:
zeros_new = zeros.sample(frac=frac) #match no of 1's
zeros_new.shape 

(24825, 564)

In [0]:
new_df = pd.concat([ones, zeros_new], join='outer') #concat both label instances
new_df.shape #view shape

(49650, 564)

In [0]:
new_df.head() #view df

,Unnamed: 0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,REGION_RATING_CLIENT,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,EXT_SOURCE_2,EXT_SOURCE_3,YEARS_BEGINEXPLUATATION_AVG,FLOORSMAX_AVG,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,...,IN_CLIENT_IN_LOAN_NUM_INSTALMENT_NUMBER_mean_min,IN_CLIENT_IN_LOAN_NUM_INSTALMENT_VERSION_sum_min,IN_CLIENT_IN_LOAN_NUM_INSTALMENT_VERSION_sum_mean,IN_CLIENT_IN_LOAN_AMT_PAYMENT_min_min,IN_CLIENT_IN_LOAN_AMT_PAYMENT_min_mean,IN_CLIENT_IN_LOAN_AMT_PAYMENT_mean_min,IN_CLIENT_IN_LOAN_AMT_INSTALMENT_max_min,IN_CLIENT_IN_LOAN_AMT_INSTALMENT_max_mean,IN_CLIENT_IN_LOAN_AMT_PAYMENT_sum_min,IN_CLIENT_IN_LOAN_AMT_PAYMENT_sum_mean,CASH_CLIENT_CASH_LOAN_MONTHS_BALANCE_sum_min,CASH_CLIENT_CASH_LOAN_INSTALLMENTS_PAID_min_min,CASH_CLIENT_CASH_LOAN_SK_DPD_DEF_min_mean,CASH_CLIENT_CASH_LOAN_NAME_CONTRACT_STATUS_Canceled_mean_min,CASH_CLIENT_CASH_LOAN_NAME_CONTRACT_STATUS_Amortized debt_mean_min,CASH_CLIENT_CASH_LOAN_NAME_CONTRACT_STATUS_XNA_mean_mean,CASH_CLIENT_CASH_LOAN_NAME_CONTRACT_STATUS_Canceled_mean_mean,CASH_CLIENT_CASH_LOAN_NAME_CONTRACT_STATUS_Demand_mean_min,CASH_CLIENT_CASH_LOAN_NAME_CONTRACT_STATUS_Approved_mean_min,CASH_CLIENT_CASH_LOAN_NAME_CONTRACT_STATUS_Approved_mean_mean,CASH_CLIENT_CASH_LOAN_NAME_CONTRACT_STATUS_Returned to the store_mean_min,CASH_CLIENT_CASH_LOAN_SK_DPD_DEF_mean_min,CASH_CLIENT_CASH_LOAN_SK_DPD_mean_min,CASH_CLIENT_CASH_LOAN_INSTALLMENTS_PAID_min_mean,CASH_CLIENT_CASH_LOAN_INSTALLMENTS_PAID_min_max,CASH_CLIENT_CASH_LOAN_SK_DPD_DEF_mean_mean,CASH_CLIENT_CASH_LOAN_SK_DPD_DEF_sum_mean,CASH_CLIENT_CASH_LOAN_NAME_CONTRACT_STATUS_Returned to the store_mean_mean,CASH_CLIENT_CASH_LOAN_NAME_CONTRACT_STATUS_Signed_mean_min,CASH_CLIENT_CASH_LOAN_NAME_CONTRACT_STATUS_Signed_mean_mean,CASH_CLIENT_CASH_LOAN_CNT_INSTALMENT_FUTURE_min_min,CASH_CLIENT_CASH_LOAN_CNT_INSTALMENT_FUTURE_min_mean,CASH_CLIENT_CASH_LOAN_NAME_CONTRACT_STATUS_Completed_mean_min,CASH_CLIENT_CASH_LOAN_NAME_CONTRACT_STATUS_Completed_mean_mean,CASH_CLIENT_CASH_LOAN_NAME_CONTRACT_STATUS_Active_mean_min,CASH_CLIENT_CASH_LOAN_INSTALLMENTS_PAID_mean_mean,CASH_CLIENT_CASH_LOAN_CNT_INSTALMENT_FUTURE_mean_min,CASH_CLIENT_CASH_LOAN_CNT_INSTALMENT_FUTURE_mean_mean,CASH_CLIENT_CASH_LOAN_CNT_INSTALMENT_min_min,CASH_CLIENT_CASH_LOAN_CNT_INSTALMENT_min_mean
0,0,100002,1.0,0.0,202500.0,406597.5,24700.5,0.018801,-9461.0,-637.0,-3648.0,-2120.0,1.0,1.0,0.0,1.0,1.0,0.0,2.0,10.0,0.0,0.0,0.0,0.0,0.262949,0.139376,0.9722,0.0833,0.0149,2.0,2.0,-1134.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,20.0,20.0,9251.775,9251.775,11559.247105,53093.745,53093.745,219625.695,219625.695,-190.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,6.0,0.000000,0.000000,1.000000,9.000000,15.000000,15.000000,24.0,24.0
26,26,100031,1.0,0.0,112500.0,979992.0,27076.5,0.018029,-18724.0,-2628.0,-6573.0,-1827.0,1.0,1.0,0.0,1.0,0.0,0.0,3.0,9.0,0.0,0.0,0.0,0.0,0.548477,0.190706,0.9732,0.0417,0.0085,10.0,1.0,-161.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,40,100047,1.0,0.0,202500.0,1193580.0,35028.0,0.025164,-17482.0,-1262.0,-1182.0,-1029.0,1.0,1.0,0.0,1.0,0.0,0.0,2.0,9.0,0.0,0.0,0.0,0.0,0.306841,0.320163,0.9960,0.2500,0.1463,0.0,0.0,-1075.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.5,10.0,10.5,4577.940,7847.820,4585.999500,4586.895,63607.590,45859.995,134273.790,-891.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

# **Split train and test set**

In [0]:
#shuffle and stratify data
#split 80:20

X_train, X_test, y_train, y_test = train_test_split(new_df.drop(["TARGET"],axis=1), new_df[["TARGET"]].values.ravel(), stratify = new_df[["TARGET"]].values.ravel(), shuffle=True, random_state=42, train_size=0.8) # 75:25

In [0]:
train_ids=X_train["SK_ID_CURR"] #retain ID col from train
test_ids=X_test["SK_ID_CURR"] #retain ID col from test
X_train=X_train.drop("SK_ID_CURR",axis=1) #drop ID col from train
X_test=X_test.drop("SK_ID_CURR",axis=1) #drop ID col from test

# **Impute missing values**

In [0]:

# Median imputation of missing values
imputer = SimpleImputer(strategy = 'median', missing_values = np.nan)

# Fit on the train data
imputer.fit(X_train)

# Transform on train and test data
X_train = imputer.transform(X_train)
X_test = imputer.transform(X_test)


# **Scale values**

In [0]:

# Scale each feature to 0-1
scaler = MinMaxScaler(feature_range = (0, 1))

# Fit on the train data
scaler.fit(X_train)

# Transform on the train and test data
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


In [0]:
print('Training data shape: ', X_train.shape)
print('Testing data shape: ', X_test.shape)

Training data shape:  (39720, 562)
Testing data shape:  (9930, 562)


# **Model I : Logistic Regression**

In [0]:

# Make the model with the specified regularization parameter
logmodel = LogisticRegression(C = 0.0001) #smaller values, less chance of overfitting

# Fit on the train data
logmodel.fit( X_train, y_train)


LogisticRegression(C=0.0001, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
#predict
logmodel_pred = logmodel.predict(X_test) 

**Confusion Matrix and Metrics -  Accuracy, F-1 score**

In [0]:
def metric(cm): #custom function to compute metrics
  TPR=cm[1][1]/(cm[1][0]+cm[1][1])
  FNR=cm[1][0]/(cm[1][0]+cm[1][1])
  TNR=cm[0][0]/(cm[0][0]+cm[0][1])
  FPR=cm[0][1]/(cm[0][0]+cm[0][1])
  return [TPR,FNR,TNR,FPR]

In [0]:

cm = confusion_matrix(logmodel_pred, y_test)
acc = accuracy_score(logmodel_pred, y_test)
print("The confusion matrix is:\n", cm)
print("the accuracy is:", acc)
metric(cm)
print(classification_report(rfmodel_pred, y_test)) #print f-1 score

The confusion matrix is:
 [[3311 1635]
 [1654 3330]]
the accuracy is: 0.6687814702920443
              precision    recall  f1-score   support

         0.0       0.70      0.70      0.70      4905
         1.0       0.71      0.70      0.70      5025

    accuracy                           0.70      9930
   macro avg       0.70      0.70      0.70      9930
weighted avg       0.70      0.70      0.70      9930



# **Model II: Random Forest** 

In [0]:

# Make the random forest classifier
rfmodel = RandomForestClassifier(criterion='gini', n_estimators = 200, bootstrap=True, max_depth=8, min_samples_leaf=4, random_state = 42, verbose = 1, n_jobs = -1) #parallelize jobs over all CPU's

# Fit on the train data
rfmodel.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:    8.7s finished


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=8, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=-1, oob_score=False, random_state=42, verbose=1,
                       warm_start=False)

In [0]:
#predict
rfmodel_pred = rfmodel.predict(X_test)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:    0.1s finished


**Confusion Matrix and Metrics -  Accuracy, F-1 score**

In [0]:
cm = confusion_matrix(rfmodel_pred, y_test)
acc = accuracy_score(rfmodel_pred, y_test)
print("The confusion matrix is:\n", cm)
print("the accuracy is:", acc)
metric(cm)
print(classification_report(rfmodel_pred, y_test)) #print f-1 score

The confusion matrix is:
 [[3410 1479]
 [1555 3486]]
the accuracy is: 0.6944612286002014
              precision    recall  f1-score   support

         0.0       0.69      0.70      0.69      4889
         1.0       0.70      0.69      0.70      5041

    accuracy                           0.69      9930
   macro avg       0.69      0.69      0.69      9930
weighted avg       0.69      0.69      0.69      9930



# **Model III: Light Gradient Boost**

In [0]:
#build model with hyperparameters
lgbmodel = lgb.LGBMClassifier(n_estimators=100, learning_rate=0.05, boosting_type='gbdt', random_state= 42, njobs = -1)

In [0]:
#fit on train data
lgbmodel.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.05, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, njobs=-1, num_leaves=31,
               objective=None, random_state=42, reg_alpha=0.0, reg_lambda=0.0,
               silent=True, subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)

In [0]:
#predict
lgbmodel_pred = lgbmodel.predict(X_test)

**Confusion Matrix and Metrics -  Accuracy, F-1 score**

In [0]:
cm = confusion_matrix(lgbmodel_pred, y_test)
acc = accuracy_score(lgbmodel_pred, y_test)
print("The confusion matrix is:\n", cm)
print("the accuracy is:", acc)
metric(cm)
print(classification_report(rfmodel_pred, y_test)) #print f-1 score

The confusion matrix is:
 [[3489 1411]
 [1476 3554]]
the accuracy is: 0.7092648539778449
              precision    recall  f1-score   support

         0.0       0.69      0.70      0.69      4889
         1.0       0.70      0.69      0.70      5041

    accuracy                           0.69      9930
   macro avg       0.69      0.69      0.69      9930
weighted avg       0.69      0.69      0.69      9930



**Use stopping criteria to decide optimal no. of estimators**

In [0]:
train_set = lgb.Dataset(data = X_train, label = y_train) #create train lgb object
test_set = lgb.Dataset(data = X_test, label = y_test) #create test lgb object

In [0]:
default_params = lgbmodel.get_params() #obtain default parameters (with no. of estimators = 100)

# Remove the default number of estimators 
del default_params['n_estimators']

N_FOLDS = 5 #set no. of folds

# Cross validation with early stopping - stop if metric does not improve over 100 rounds
cv_results = lgb.cv(default_params, train_set, num_boost_round = 10000, early_stopping_rounds = 100, 
                    metrics = 'auc', nfold = N_FOLDS, seed = 42, stratified =True) #stratify data

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:741: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))


In [0]:
s=len(cv_results['auc-mean']) 
print(s) #this is the optimal no. of estimators to be set for lgb model

297


**Rerun model to observe changes**

In [0]:
lgbmodel2 = lgb.LGBMClassifier(n_estimators=s, learning_rate=0.05, boosting_type='gbdt', random_state= 42, njobs = -1)
lgbmodel2.fit(X_train, y_train) #fit model on train
lgbmodel2_pred = lgbmodel2.predict(X_test) #make pred

In [0]:
cm = confusion_matrix(lgbmodel2_pred, y_test) #print confusion matrix
acc = accuracy_score(lgbmodel2_pred, y_test)
print("The confusion matrix is:\n", cm)
print("the accuracy is:", acc)
metric(cm)
print(classification_report(rfmodel_pred, y_test)) #print f-1 score

The confusion matrix is:
 [[3481 1374]
 [1484 3591]]
the accuracy is: 0.7121852970795569
              precision    recall  f1-score   support

         0.0       0.69      0.70      0.69      4889
         1.0       0.70      0.69      0.70      5041

    accuracy                           0.69      9930
   macro avg       0.69      0.69      0.69      9930
weighted avg       0.69      0.69      0.69      9930



# **Model IV: Deep Neural Network**

In [0]:
#check shapes of all splits
print(X_train.shape)  
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(39720, 562)
(9930, 562)
(39720,)
(9930,)


In [0]:
#build model with many dense layers
model = tf.keras.Sequential([
        #Input layer
        tf.keras.layers.Dense(units=512, activation='relu', input_shape=(562,)), #set shape
        tf.keras.layers.Dropout(0.5), #to avoid overfitting
        #Hidden layer I
        tf.keras.layers.Dense(units=128, activation='relu'),
        #Ouput Layer
        tf.keras.layers.Dense(units=2, activation='softmax') #activation function to output probabilities '1' or '0'
       
        ])

In [0]:
# Compile Model
model.compile(loss = 'binary_crossentropy', optimizer="adam", metrics=['accuracy']) #Gradient descent with adaptive learning rate

In [0]:
# Print Model Summary 
model.summary()

#set no. of iterations
epoch = 25

#fit model on train data
history = model.fit(X_train, y_train, epochs=epoch)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               288256    
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 128)               65664     
_________________________________________________________________
dense_12 (Dense)             (None, 2)                 258       
Total params: 354,178
Trainable params: 354,178
Non-trainable params: 0
_________________________________________________________________
Epoch 1/25
1242/1242 [==============================] - 4s 3ms/step - loss: 0.6936 - accuracy: 0.4987
Epoch 2/25
1242/1242 [==============================] - 4s 3ms/step - loss: 0.6932 - accuracy: 0.4992
Epoch 3/25
1242/1242 [====================

In [0]:
#evaluate metrics on test data
model.evaluate(X_test, y_test) #accuracy is 0.5 - as good as guessing randomly !

311/311 [==============================] - 1s 2ms/step - loss: 0.6931 - accuracy: 0.5000


[0.6931459903717041, 0.5]

**Convolution Layer** 

In [0]:
model2 = tf.keras.models.Sequential([
  # 1D Convolution layer
  tf.keras.layers.Convolution1D(16, kernel_size=20,strides=1, activation='relu', input_shape=(None, 562)), #give shape
  # 1D Maxpooling
  tf.keras.layers.MaxPooling1D(pool_size=2, strides=1), #set window size and stride
  tf.keras.layers.Dropout(0.5), #to avoid overfitting
  # Hidden Layer
  tf.keras.layers.Dense(16, activation='relu', input_shape = (562,)), 
  # Output Layer
  tf.keras.layers.Dense(2, activation='softmax')
])

In [0]:
# Compile Model
model2.compile(loss = 'binary_crossentropy', optimizer="adam", metrics=['accuracy'])

In [0]:
# Print Model Summary 
model2.summary()

#set no. of iterations
epoch = 10

#fit model on train data
history = model2.fit(X_train, y_train, epochs=epoch)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, None, 16)          179856    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, None, 16)          0         
_________________________________________________________________
dropout_5 (Dropout)          (None, None, 16)          0         
_________________________________________________________________
dense_13 (Dense)             (None, None, 16)          272       
_________________________________________________________________
dense_14 (Dense)             (None, None, 2)           34        
Total params: 180,162
Trainable params: 180,162
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10


ValueError: ignored

In [0]:
#evaluate metrics on test data
model2.evaluate(X_test, y_test) #accuracy is ?